## Prozorro API

- https://github.com/openprocurement/openprocurement.api/blob/master/docs/source/tenders.rst
- https://prozorro-api-docs.readthedocs.io/en/master/authentication.html#api-keys
- [basic rules of using Prozorro API](http://api-docs.openprocurement.org/en/latest/tutorial.html)
- [Python client](https://github.com/openprocurement/openprocurement.client.python/tree/master/openprocurement_client)

In [1]:
#імпортування необхідних бібліотек
from IPython.display import JSON 
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import urllib.parse as url
#import subprocess #для Ipython notebook

In [2]:
#mount your Google Drive files
from pathlib import Path
GDRIVE_PATH='/content/gdrive/My Drive/code/Prozorro/'
if not Path(GDRIVE_PATH).exists():
    from google.colab import drive
    drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [14]:
# Print json in collapsible format in ipython (for ipython notebook)
# https://www.reddit.com/r/IPython/comments/34t4m7/lpt_print_json_in_collapsible_format_in_ipython/
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

## Робота з API Prozorro.

In [ ]:
#endpoint = 'http://lb-api-sandbox.prozorro.gov.ua/api/2.5' Prozorro sandbox API

In [8]:
#endpoint = 'http://public.api.openprocurement.org/api/0' 

endpoint = 'https://public.api.openprocurement.org/api/2.3'

In [9]:
t = datetime.now()

In [10]:
url.quote(t.isoformat())

'2020-06-18T18%3A44%3A13.767918'

In [11]:
#r = requests.get(endpoint + '/tenders?offset=' + url.quote(t.isoformat()))
r = requests.get(endpoint + '/tenders', params={
    'offset': t.isoformat(),
    'descending': 1,
})
if r.status_code != 200:
    print('Request failed, status_code=', r.status_code)

rd = r.json()

In [12]:
r.url

'https://public.api.openprocurement.org/api/2.3/tenders?offset=2020-06-18T18%3A44%3A13.767918&descending=1'

In [15]:
RenderJSON(rd)

In [16]:
tender_ids = [tender['id'] for tender in rd['data']]
tender_ids

['de7e5674f9084b74805fdd1acb8ec5c2',
 '2d9367fcff494c4ab692571c3f1527d4',
 '20a1f563319f44dca937baa8393708dd',
 '95af9cb94d624cc3afec35a8b268fab6',
 'd87f5fc6d7464166ad6adb4707cad367',
 '185e0785b6664cafa0079d4feaa75289',
 '36e0ddce97594b409b4c8f6bec08f4fe',
 '5ff9b435c8be46ab80f121093af948ff',
 'd571d13df5414ee7850c54959ec95b8d',
 'eb6ec6b4852e4bffbff6bec7df589673',
 '1f0756ae689e4de39d141f685962516e',
 '978bdf8895c74c788a2f0b173a999395',
 '414f391498654127ac9df5cedfb3bd34',
 '5a194743e32c42efa400fecdeacfdc18',
 '80f1ec5be7c84bf7a392ab98cf904f96',
 '9837b847e4b74484a5b568b277e0709c',
 '74243b74f63e4ce7a7acb48b2ab29e90',
 '4efef7b30fcf49288a58208219323dcf',
 '1bf3b0b2ad2c4014883cc409d3d73295',
 '361ba152a2e04287a86b10f54d5e7541',
 '82017d4dbe934f5a914726e7d3383fa6',
 'd7806c239ddc446e8767710f3f403f0b',
 'a1f363919d8845b1b5827f06e98bf07e',
 'c1e9f94377324a199e58fbe3b6ecd895',
 '2add2d0f253f46f59b5db73ba2b01d14',
 'b246552695554e038d30ab3123244c22',
 '0cac15afcea14b3f88ace226b1833f7e',
 

## Приклад витягування даних про конкретний тендер

In [19]:
tender_id = 'ee53a79b3fab419a90c1dbada962999a' 
#tender_id = rd['data'][8]['id']

In [20]:
r1 = requests.get(f'{endpoint}/tenders/{tender_id}')
print('status', r1.status_code)

status 200


In [21]:
RenderJSON(r1.json()) # для Ipython notebook

In [22]:
rd1 = r1.json()['data']
tender_title = rd1['title']
for item in rd1['items']:
    clss = item['classification']
    print(f"tender_title={tender_title}, item_desc={item['description']}")
    print(f"classification={clss['scheme']} {clss['id']}")
    print()

tender_title=Послуги із збирання, зберігання, перевезення (транспортування) та оброблення шляхом знешкодження небезпечних відходів, item_desc=Послуги із збирання, зберігання, перевезення (транспортування) та оброблення шляхом знешкодження небезпечних відходів
classification=ДК021 90520000-8



## Витягування всіх тендерів

In [23]:
tenders = []

for tender_id in tender_ids:
    r = requests.get(f'{endpoint}/tenders/{tender_id}')
    if r.status_code != 200:
        print('status', r.status_code)
        continue
    data = r.json()['data']
    tenders.append(data)
    #tender_title = data['title']
    #tender_desc = data.get('description')

In [24]:
tenders

Output hidden; open in https://colab.research.google.com to view.

In [25]:
tenders[0].keys()

dict_keys(['procurementMethod', 'numberOfBids', 'awardPeriod', 'mainProcurementCategory', 'complaintPeriod', 'auctionUrl', 'enquiryPeriod', 'submissionMethod', 'complaints', 'procuringEntity', 'owner', 'id', 'plans', 'documents', 'title', 'tenderID', 'guarantee', 'dateModified', 'status', 'tenderPeriod', 'contracts', 'auctionPeriod', 'procurementMethodType', 'awards', 'date', 'milestones', 'minimalStep', 'items', 'bids', 'value', 'awardCriteria'])

In [26]:
tenders[0]['items']

[{'classification': {'description': 'Мережеві кабелі',
   'id': '31310000-2',
   'scheme': 'ДК021'},
  'deliveryAddress': {'countryName': 'Україна',
   'locality': 'Новогродівка',
   'postalCode': '85483',
   'region': 'Донецька область',
   'streetAddress': 'відокремлений підрозділ «Шахта 1/3 «Новогродівська» Державного підприємства «Селидіввугілля» вул.Шахтна,1'},
  'deliveryDate': {'endDate': '2020-12-31T00:00:00+02:00'},
  'description': 'МЕРЕЖЕВІ КАБЕЛІ (кабель ВЕВБбШв 6 3*95+1*10 (або еквівалент))',
  'id': '5bc512da5f6943dfb931370b3478b81f',
  'quantity': 1500.0,
  'unit': {'code': 'MTR', 'name': 'метри'}}]

In [27]:
tenders[0]['tenderID']

'UA-2020-05-27-006838-b'

In [28]:
titles = []
descs = []
idescs = []
iunits = []
iclasses = []
ids = []
tenderIDs = []
 
for data in tenders:
    for item in data['items']:
        idescs.append(item['description'])
        iunits.append(item['unit']['name'])
        iclasses.append(item['classification']['description']+' '+item['classification']['id'])
        titles.append(data['title'])
        descs.append(data.get('description'))
        ids.append(data['id'])
        tenderIDs.append(data['tenderID'])

df = pd.DataFrame({
    'id': ids,
    'tenderID': tenderIDs,
    'title': titles,
    'description': descs, 
    'item_desc': idescs, 
    'item_unit': iunits,
    'item_class': iclasses,
})

df.head(20)

,id,tenderID,title,description,item_desc,item_unit,item_class
0,de7e5674f9084b74805fdd1acb8ec5c2,UA-2020-05-27-006838-b,Мережеві кабелі (кабель ВЕВБбШв 6 3*95+1*10 (а...,None,МЕРЕЖЕВІ КАБЕЛІ (кабель ВЕВБбШв 6 3*95+1*10 (а...,метри,Мережеві кабелі 31310000-2
1,2d9367fcff494c4ab692571c3f1527d4,UA-2020-06-18-008554-c,Послуги надання доступу до всесвітньої мережі ...,"Найменування замовника: АТ «Укрпошта», 01001, ...",Послуги надання доступу до всесвітньої мережі ...,послуга,Послуги телефонного зв’язку та передачі даних ...
2,20a1f563319f44dca937baa8393708dd,UA-2020-05-29-005484-b,ДК 021:2015:71310000-4 Консультаційні послуги ...,"Місцезнаходження замовника:10014, Житомирська ...","""Проведення енергетичних аудитів будівель соці...",послуга,Консультаційні послуги у галузях інженерії та ...
3,95af9cb94d624cc3afec35a8b268fab6,UA-2020-06-18-002538-a,"Свічки запалення ГАЗ-2705, свічки запалення ЗІ...",None,Свічки запалення ГАЗ-2705,шт,Двигуни та їх частини 34310000-3
4,95af9cb94d624cc3afec35a8b268fab6,UA-2020-06-18-002538-a,"Свічки запалення ГАЗ-2705, свічки запалення ЗІ...",None,свічки запалення ЗІЛ-131,шт,Двигуни та їх частини 34310000-3
5,95af9cb94d624cc3afec35a8b268fab6,UA-2020-06-18-002538-a,"Свічки запалення ГАЗ-2705, свічки запалення ЗІ...",None,пас приводу генератора КрАЗ,шт,Двигуни та їх частини 34310000-3
6,95af9cb94d624cc3afec35a8b268fab6,UA-2020-06-18-002538-a,"Свічки запалення ГАЗ-2705, свічки запалення ЗІ...",None,кришка радіатора ЗІЛ,шт,Двигуни та їх частини 34310000-3
7,d87f5fc6d7464166ad6adb4707cad367,UA-2020-06-18-005493-b,Противогаз,None,Противогаз,шт,"Протипожежне, рятувальне та захисне обладнання..."
8,185e0785b6664cafa0079d4feaa75289,UA-2020-06-18-005469-b,ДК 021:2015 – 44420000-0 «Будівельні товари» (...,None,ДК 021:2015 – 44420000-0 «Будівельні товари» (...,шт,Будівельні товари 44420000-0
9,36e0ddce97594b409b4c8f6bec08f4fe,UA-2020-06-18-002533-a,"09134200-9 - Дизельне паливо, 09132000-3 - Бензин","Категорія замовника: юридична особа, яка забез...",09134200-9 - Дизельне паливо,літр,Нафта і дистиляти 09130000-9


In [29]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html

df1 = pd.json_normalize(tenders, 'items', meta=['title', 'id', 'tenderID'], record_prefix='items.')

df1 = df1[['title', 'id', 'tenderID',
           'items.description', 'items.unit.name', 'items.unit.code', 
           'items.classification.description', 'items.classification.id',
          ]]

In [30]:
df1.sort_values(by=['id'])

,title,id,tenderID,items.description,items.unit.name,items.unit.code,items.classification.description,items.classification.id
110,Елементи електричних схем,0063cd6050bf4aaf9c878bfe685a4b8a,UA-2020-06-18-005466-b,"Сплайс-касета для 24 з'єднань, тип 01, сіра, F...",штуки,H87,Елементи електричних схем,31220000-4
119,Перезарядка картріджів,04f63aa1d59c47c5b6ead6819db2e105,UA-2020-06-18-008423-c,Перезарядка картриджів,штуки,H87,Послуги з ремонту і технічного обслуговування ...,50320000-4
138,39510000-0 Вироби домашнього текстилю (ролетні...,05450c93e88d43f38e71154944f6270a,UA-2020-06-18-008546-c,39510000-0 Вироби домашнього текстилю (ролетні...,штуки,H87,Вироби домашнього текстилю,39510000-0
111,Товари господарського призначення,0663ef5a8e9e46cc8324ca3cc7f9a187,UA-2020-06-18-005497-b,Змішувач ІРІДА кухня хром,штуки,H87,Машини спеціального призначення різні,42990000-2
137,Агрохімічна продукція,06dae1cfb6964e08bbc80fe178454a8d,UA-2020-06-18-008538-c,Винсепт,штуки,H87,Агрохімічна продукція,24450000-3
...,...,...,...,...,...,...,...,...
74,Пневматичний інструмент,f70f13a41f924ff587075ed0595a88df,UA-2020-05-29-000241-b,Пневматичний інструмент,шт,H87,Ручні інструменти пневматичні чи моторизовані,42650000-7
136,Капітальний ремонт Шевченківського закладу заг...,fb9cf5fc3cca4df7a7ae57ca8393b697,UA-2020-05-05-003572-b,Капітальний ремонт Шевченківського закладу заг...,роботи,E51,Капітальний ремонт і реставрація,45453000-7
102,дріжджі,fe65a25a4a3b43888fface3e60910e67,UA-2020-06-18-002536-a,"дріжджі хлібні ""Харківські""",штуки,H87,Продукти харчування та сушені продукти різні,15890000-3
101,дріжджі,fe65a25a4a3b43888fface3e60910e67,UA-2020-06-18-002536-a,дріжджі сухі,штуки,H87,Продукти харчування та сушені продукти різні,15890000-3


## Робота з датафремом запланованих закупок

In [31]:
#завантаження даних
Plans = pd.read_excel(GDRIVE_PATH+'/Plans-vs-OCDS.xlsx')

In [32]:
#перегляд перших 5 рядків df
Plans.head()

,Ідентифікатор пункт плану,Пункт плану (розширений),Примітки,Сума пунктів плану,Процедура закупівлі,...,Назва організатора,Рік-Місяць планованого оголошення,Рік-Місяць доставки,К-сть місяців з публікації до оголошення,Дата публікації пункту плану,Класифікація CPV
0,UA-P-2020-04-28-000015-b,інженерно-геологічних вишукувань по об’єкту: «...,-,4900.0,Відкриті торги,https://prozorro.gov.ua/plan/UA-P-2020-04-28-0...,"Меморіальний музей тоталітарних режимів ""Терит...",2020-кві,-,0,2020-04-28,71510000-6 Послуги у сфері інженерно-геологічн...
1,UA-P-2020-04-27-004544-b,Послуги: Поточний ремонт ливневої каналізаційн...,Послуги: Поточний ремонт ливневої каналізаційн...,49046.0,Звіт про укладений договір,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,Личаківська районна адміністрація Львівської м...,2020-кві,-,0,2020-04-27,"45230000-8 Будівництво трубопроводів, ліній зв..."
2,UA-P-2020-04-27-004528-b,"""Капітальний ремонт даху з водовідведенням у ...",-,9947.0,Звіт про укладений договір,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,Ліцей № 51 ім. Івана Франка Львівської міської...,2020-кві,-,0,2020-04-27,71520000-9 Послуги з нагляду за виконанням буд...
3,UA-P-2020-04-27-004503-b,ДК 021:2015 — 50112000-3 - Послуги з ремонту і...,-,6220.0,Звіт про укладений договір,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,Сихівська районна адміністрація Львівської міс...,2020-кві,-,0,2020-04-27,50112000-3 Послуги з ремонту і технічного обсл...
4,UA-P-2020-04-27-004470-b,"""Ремонтно-реставраційні роботи будинку на вул....","Категорія замовника. підприємства, установи, о...",9970759.2,Відкриті торги,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,"Львівське комунальне підприємство ""Агенція рес...",2020-кві,-,0,2020-04-27,45453000-7 Капітальний ремонт і реставрація


### Відфільтровування описів тендерів, які містять цільове значення 

In [ ]:
# Examples
s1 = pd.Series(['Mouse', 'dog', 'house and parrot', 23, np.NaN])
s1.str.contains('og', regex=False)

pd.Series(['foo', 'fuz', np.nan]).str.replace('f.', 'ba', regex=True)

0    bao
1    baz
2    NaN
dtype: object

In [ ]:
Plans.columns

Index(['Ідентифікатор пункт плану', 'Пункт плану (розширений)', 'Примітки',
       'Сума пунктів плану', 'Процедура закупівлі', '...',
       'Назва організатора', 'Рік-Місяць планованого оголошення',
       'Рік-Місяць доставки', 'К-сть місяців з публікації до оголошення',
       'Дата публікації пункту плану', 'Класифікація CPV'],
      dtype='object')

In [ ]:
Plans.shape

(15384, 12)

In [ ]:
Plans['plans_with_code'] = np.nan

In [ ]:
Plans.loc[Plans['Пункт плану (розширений)'].str.contains('\d{8}\D\d'), 'plans_with_code'] = 1

In [ ]:
Plans.isnull().sum()
#mask = Plans[['Пункт плану (розширений)']].apply(lambda x: x.str.contains('\d{8}[-]\d', regex=True)).any(axis=1)
#Plans['Пункт плану (розширений)'][mask]

Ідентифікатор пункт плану                       0
Пункт плану (розширений)                        0
Примітки                                     2023
Сума пунктів плану                              0
Процедура закупівлі                             0
...                                             0
Назва організатора                              0
Рік-Місяць планованого оголошення               0
Рік-Місяць доставки                             0
К-сть місяців з публікації до оголошення        0
Дата публікації пункту плану                    0
Класифікація CPV                                0
plans_with_code                             14550
dtype: int64

In [ ]:
Plans['Пункт плану (розширений)'] = Plans['Пункт плану (розширений)'].str.replace('\d{8}\D\d', '', regex=True)\
                                    .replace('\w{2} \d{3}:\d{4}', '', regex=True)\
                                    .replace('[|]', '', regex=True)\
                                    .replace('[-—:]', '', regex=True)

In [ ]:
Plans.head(5)

,Ідентифікатор пункт плану,Пункт плану (розширений),Примітки,Сума пунктів плану,Процедура закупівлі,...,Назва організатора,Рік-Місяць планованого оголошення,Рік-Місяць доставки,К-сть місяців з публікації до оголошення,Дата публікації пункту плану,Класифікація CPV,plans_with_code
0,UA-P-2020-04-28-000015-b,інженерногеологічних вишукувань по об’єкту «Ре...,-,4900.0,Відкриті торги,https://prozorro.gov.ua/plan/UA-P-2020-04-28-0...,"Меморіальний музей тоталітарних режимів ""Терит...",2020-кві,-,0,2020-04-28,71510000-6 Послуги у сфері інженерно-геологічн...,NaN
1,UA-P-2020-04-27-004544-b,Послуги Поточний ремонт ливневої каналізаційно...,Послуги: Поточний ремонт ливневої каналізаційн...,49046.0,Звіт про укладений договір,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,Личаківська районна адміністрація Львівської м...,2020-кві,-,0,2020-04-27,"45230000-8 Будівництво трубопроводів, ліній зв...",1.0
2,UA-P-2020-04-27-004528-b,"""Капітальний ремонт даху з водовідведенням у ...",-,9947.0,Звіт про укладений договір,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,Ліцей № 51 ім. Івана Франка Львівської міської...,2020-кві,-,0,2020-04-27,71520000-9 Послуги з нагляду за виконанням буд...,NaN
3,UA-P-2020-04-27-004503-b,Послуги з ремонту і технічного обслуговува...,-,6220.0,Звіт про укладений договір,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,Сихівська районна адміністрація Львівської міс...,2020-кві,-,0,2020-04-27,50112000-3 Послуги з ремонту і технічного обсл...,1.0
4,UA-P-2020-04-27-004470-b,"""Ремонтнореставраційні роботи будинку на вул. ...","Категорія замовника. підприємства, установи, о...",9970759.2,Відкриті торги,https://prozorro.gov.ua/plan/UA-P-2020-04-27-0...,"Львівське комунальне підприємство ""Агенція рес...",2020-кві,-,0,2020-04-27,45453000-7 Капітальний ремонт і реставрація,NaN


In [ ]:
Plans.isnull().sum()

Ідентифікатор пункт плану                       0
Пункт плану (розширений)                        0
Примітки                                     2023
Сума пунктів плану                              0
Процедура закупівлі                             0
...                                             0
Назва організатора                              0
Рік-Місяць планованого оголошення               0
Рік-Місяць доставки                             0
К-сть місяців з публікації до оголошення        0
Дата публікації пункту плану                    0
Класифікація CPV                                0
plans_with_code                             14550
dtype: int64

In [ ]:
NUM_PRED = 10

In [ ]:
template_q = '''
    {
      Predictions {
        Classification(
          page: {limit: %d},
          filters:[
          %%s
          ]){
          values{
            entity{
              id
              description
              scheme
            }
            probability
          }
        }
      }
    }
''' % NUM_PRED

filters = '''
{eq: {field: "tender.title", value: "Плитка керамічна"}},
{eq: {field: "tender.description", value: "плитка керамічна"}},
{eq: {field: "item.description", value: "Плитка керамічна Gress Атем Veneto, для підлоги, розмір – 400*400, колір – бежева, поверхня глазурована, матова"}},
{eq: {field: "item.unit.id", value: "MTK"}},
{gte: {field: "probability", value: "0.0002"}}'''

print(template_q % filters)


    {
      Predictions {
        Classification(
          page: {limit: 10},
          filters:[
          
{eq: {field: "tender.title", value: "Плитка керамічна"}},
{eq: {field: "tender.description", value: "плитка керамічна"}},
{eq: {field: "item.description", value: "Плитка керамічна Gress Атем Veneto, для підлоги, розмір – 400*400, колір – бежева, поверхня глазурована, матова"}},
{eq: {field: "item.unit.id", value: "MTK"}},
{gte: {field: "probability", value: "0.0002"}}
          ]){
          values{
            entity{
              id
              description
              scheme
            }
            probability
          }
        }
      }
    }



In [ ]:
def generate_filters(tender_desc, tender_title=None, item_desc=None, item_unit=None, prob=0.0002):
    fs = []
    def sanitize(s):
        return s.replace('"', '«').replace('\n', ' ').replace('\\', r'\\')
    
    if tender_title is not None:
        fs.append('{eq: {field: "tender.title", value: "%s"}}' % sanitize(tender_title))
    
    fs.append('{eq: {field: "tender.description", value: "%s"}}' % sanitize(tender_desc))
    
    if item_desc is not None:
        fs.append('{eq: {field: "item.description", value: "%s"}}' % sanitize(item_desc))
        
    if item_unit is not None:
        fs.append('{eq: {field: "item.unit.id", value: "%s"}}' % sanitize(item_unit))
        
    fs.append('{gte: {field: "probability", value: "%f"}}' % prob)
    return ",\n".join(fs)

print(generate_filters('плитка керамічна', 
                       tender_title='Плитка керамічна',
                       item_desc='Плитка керамічна Gress Атем "Veneto", для підлоги, розмір – 400*400, колір – бежева, поверхня глазурована, матова'))
    
        

{eq: {field: "tender.title", value: "Плитка керамічна"}},
{eq: {field: "tender.description", value: "плитка керамічна"}},
{eq: {field: "item.description", value: "Плитка керамічна Gress Атем «Veneto«, для підлоги, розмір – 400*400, колір – бежева, поверхня глазурована, матова"}},
{gte: {field: "probability", value: "0.000200"}}


In [ ]:
#only for colab
!pip3 install gql

     |████████████████████████████████| 256kB 3.9MB/s 
     |████████████████████████████████| 184kB 20.0MB/s 


In [ ]:
#use only for colab
from getpass import getpass 

In [ ]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
from json import dumps
base_url = 'https://ocdsanalytics.com/ua/prozorro/graphql'

#for ipython notebook:
#with open('./graphql_token') as f:
#    auth_token = f.read().strip()

#for colab:
auth_token = getpass('Your graphql token, please: ')

transport_ocds = RequestsHTTPTransport(url = base_url, 
                                       headers = {'Authorization':'Bearer '+auth_token}, 
                                       use_json=True)

client = Client(transport=transport_ocds)

Your graphql token, please: ··········


In [ ]:
data = []

In [ ]:
import time 

print('before')
time.sleep(5)
print('after')

before
after


In [ ]:
def flatten_entities(r):
    d = {}
    def flatten(i):
        d['id'+str(i+1)] = r[i]['entity']['id']
        #d['desc'+str(i+1)] = r[i]['entity']['description']
        d['acc'+str(i+1)] = r[i]['probability']

    for i in range(0, min(NUM_PRED, len(r))):
        flatten(i)

    return d

In [ ]:
flatten_entities([
        {'entity': {
                'id': '50112000-3', 
                'description': 'Послуги з ремонту і технічного обслуговування автомобілів', 
                'scheme': 'CPV'},
         'probability': 0.9890542}, 
        {'entity': {
                'id': '50110000-9', 
                'description': 'Послуги з ремонту і технічного обслуговування мототранспортних засобів і супутнього обладнання',
                'scheme': 'CPV'},
         'probability': 0.010862832}
    ])

{'acc1': 0.9890542,
 'acc2': 0.010862832,
 'id1': '50112000-3',
 'id2': '50110000-9'}

In [ ]:
%%time
params = {'limit':1} 

#try:
#for i in range(50): #передавання лише перших n-рядків з Plans для тестування;
for i in range(len(data), len(Plans)): #передавання всіх рядків в Plans
    tender_desc = Plans['Пункт плану (розширений)'][i]
    CPV_Plans = Plans['Класифікація CPV'][i]
    Plan_identificator = Plans['Ідентифікатор пункт плану'][i]
    plans_desc_with_code = Plans['plans_with_code'][i]
    
    q = template_q % generate_filters(tender_desc)
    print(q)
    q = gql(q)

    r = client.execute(q, variable_values=params)
    r = r['Predictions']['Classification']['values']
    row = flatten_entities(r)
    
    row['CPV з плану'] = CPV_Plans
    row['Пункт плану (розширений)'] = tender_desc
    row['Ідентифікатор пункту плану'] = Plan_identificator
    row['plans_with_code'] = plans_desc_with_code
    #print(row)

    #print(f"prob={p:0.2f}, id={r0['id']}, scheme={r0['scheme']}, desc={r0['description']}")
    data.append(row)
    time.sleep(0.2)
#except:
#    subprocess.call(['mplayer', '/home/qgelena/Music/Scary-stinger-dun-dun-dun.mp3'])
#    raise
    
#subprocess.call(['mplayer', '/home/qgelena/Music/Cat-meows-sounds.wav'])

Streaming output truncated to the last 5000 lines.
    {
      Predictions {
        Classification(
          page: {limit: 10},
          filters:[
          {eq: {field: "tender.description", value: "Постійний технічний супровід комп’ютерної програми «Єдина інформаційна система управління бюджетом«  "}},
{gte: {field: "probability", value: "0.000200"}}
          ]){
          values{
            entity{
              id
              description
              scheme
            }
            probability
          }
        }
      }
    }


    {
      Predictions {
        Classification(
          page: {limit: 10},
          filters:[
          {eq: {field: "tender.description", value: "Послуга з доступу та підтримки сервера Законодавство  "}},
{gte: {field: "probability", value: "0.000200"}}
          ]){
          values{
            entity{
              id
              description
              scheme
            }
            probability
          }
        }
      }
    }


In [ ]:
# Для ручного додавання колонки, це вже робиться в циклі;
#df1['Plan_identificator'] = Plans['Ідентифікатор пункт плану'] 
#df1['description_plan_item'] = Plans['Пункт плану (розширений)']

In [ ]:
df1 = pd.DataFrame(data)

In [ ]:
df1.columns

Index(['id1', 'acc1', 'id2', 'acc2', 'id3', 'acc3', 'id4', 'acc4', 'id5',
       'acc5', 'id6', 'acc6', 'id7', 'acc7', 'id8', 'acc8', 'id9', 'acc9',
       'id10', 'acc10', 'CPV з плану', 'Пункт плану (розширений)',
       'Ідентифікатор пункту плану', 'plans_with_code'],
      dtype='object')

In [ ]:
df1.head()

,id1,acc1,id2,acc2,id3,acc3,id4,acc4,id5,acc5,id6,acc6,id7,acc7,id8,acc8,id9,acc9,id10,acc10,CPV з плану,Пункт плану (розширений),Ідентифікатор пункту плану,plans_with_code
0,79713000-5,0.244532,79710000-4,0.065498,71240000-2,0.045981,45454000-4,0.044919,71355000-1,0.044411,45000000-7,0.043375,71510000-6,0.040920,71250000-5,0.037525,79711000-1,0.034360,71320000-7,0.033608,71510000-6 Послуги у сфері інженерно-геологічн...,інженерногеологічних вишукувань по об’єкту «Ре...,UA-P-2020-04-28-000015-b,NaN
1,45230000-8,0.999917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"45230000-8 Будівництво трубопроводів, ліній зв...",Послуги Поточний ремонт ливневої каналізаційно...,UA-P-2020-04-27-004544-b,1.0
2,45453000-7,0.412781,45450000-6,0.153498,45261000-4,0.152376,45260000-7,0.059486,45200000-9,0.047588,45400000-1,0.034377,45210000-2,0.025401,45230000-8,0.012260,45300000-0,0.012021,45000000-7,0.009533,71520000-9 Послуги з нагляду за виконанням буд...,"""Капітальний ремонт даху з водовідведенням у ...",UA-P-2020-04-27-004528-b,NaN
3,50112000-3,0.517599,50110000-9,0.414187,50530000-9,0.018717,50112200-5,0.018097,50112100-4,0.009310,50114000-7,0.005127,50116000-1,0.004195,50100000-6,0.004024,50000000-5,0.003200,63710000-9,0.000843,50112000-3 Послуги з ремонту і технічного обсл...,Послуги з ремонту і технічного обслуговува...,UA-P-2020-04-27-004503-b,1.0
4,45260000-7,0.197195,45230000-8,0.151064,45310000-3,0.091480,45330000-9,0.088683,45453000-7,0.085086,45450000-6,0.047697,45420000-7,0.029505,45261000-4,0.023586,45343100-4,0.021437,45000000-7,0.016413,45453000-7 Капітальний ремонт і реставрація,"""Ремонтнореставраційні роботи будинку на вул. ...",UA-P-2020-04-27-004470-b,NaN


In [ ]:
cols = ['plans_with_code', 'Ідентифікатор пункту плану', 'Пункт плану (розширений)', 'CPV з плану',]

for i in range(1, NUM_PRED+1):
    cols.extend(['id'+str(i), 'acc'+str(i)]) #'desc'+str(i), 

print(cols)

['plans_with_code', 'Ідентифікатор пункту плану', 'Пункт плану (розширений)', 'CPV з плану', 'id1', 'acc1', 'id2', 'acc2', 'id3', 'acc3', 'id4', 'acc4', 'id5', 'acc5', 'id6', 'acc6', 'id7', 'acc7', 'id8', 'acc8', 'id9', 'acc9', 'id10', 'acc10']


In [ ]:
df1 = df1[cols]

In [ ]:
df1.head(20)

,plans_with_code,Ідентифікатор пункту плану,Пункт плану (розширений),CPV з плану,id1,acc1,id2,acc2,id3,acc3,id4,acc4,id5,acc5,id6,acc6,id7,acc7,id8,acc8,id9,acc9,id10,acc10
0,NaN,UA-P-2020-04-28-000015-b,інженерногеологічних вишукувань по об’єкту «Ре...,71510000-6 Послуги у сфері інженерно-геологічн...,79713000-5,0.244532,79710000-4,0.065498,71240000-2,0.045981,45454000-4,0.044919,71355000-1,0.044411,45000000-7,0.043375,71510000-6,0.040920,71250000-5,0.037525,79711000-1,0.034360,71320000-7,0.033608
1,1.0,UA-P-2020-04-27-004544-b,Послуги Поточний ремонт ливневої каналізаційно...,"45230000-8 Будівництво трубопроводів, ліній зв...",45230000-8,0.999917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,UA-P-2020-04-27-004528-b,"""Капітальний ремонт даху з водовідведенням у ...",71520000-9 Послуги з нагляду за виконанням буд...,45453000-7,0.412781,45450000-6,0.153498,45261000-4,0.152376,45260000-7,0.059486,45200000-9,0.047588,45400000-1,0.034377,45210000-2,0.025401,45230000-8,0.012260,45300000-0,0.012021,45000000-7,0.009533
3,1.0,UA-P-2020-04-27-004503-b,Послуги з ремонту і технічного обслуговува...,50112000-3 Послуги з ремонту і технічного обсл...,50112000-3,0.517599,50110000-9,0.414187,50530000-9,0.018717,50112200-5,0.018097,50112100-4,0.009310,50114000-7,0.005127,50116000-1,0.004195,50100000-6,0.004024,50000000-5,0.003200,63710000-9,0.000843
4,NaN,UA-P-2020-04-27-004470-b,"""Ремонтнореставраційні роботи будинку на вул. ...",45453000-7 Капітальний ремонт і реставрація,45260000-7,0.197195,45230000-8,0.151064,45310000-3,0.091480,45330000-9,0.088683,45453000-7,0.085086,45450000-6,0.047697,45420000-7,0.029505,45261000-4,0.023586,45343100-4,0.021437,45000000-7,0.016413
5,1.0,UA-P-2020-04-27-004445-b,Антисептичні та дезінфекційні засоби,33631600-8 Антисептичні та дезінфекційні засоби,33631600-8,0.420096,33630000-5,0.190730,33600000-6,0.109936,24455000-8,0.107754,24450000-3,0.089788,33631000-2,0.033706,24400000-8,0.008875,33741300-9,0.007319,33740000-9,0.005148,33000000-0,0.003557
6,NaN,UA-P-2020-04-27-004398-b,Металопластикові вироби,"44221000-5 Вікна, двері та супутні вироби",44221000-5,0.380415,44220000-8,0.244600,44210000-5,0.088449,19520000-7,0.063580,44221100-6,0.054114,44200000-2,0.023929,44160000-9,0.023448,44310000-6,0.020361,44110000-4,0.011547,19500000-1,0.006274
7,1.0,UA-P-2020-04-27-004370-b,Нагляд за будівельними роботами ТН на капремо...,71247000-1 Нагляд за будівельними роботами,71247000-1,0.326271,45453000-7,0.187396,45000000-7,0.143045,71520000-9,0.130678,71248000-8,0.031905,71240000-2,0.024127,99999999-9,0.021558,45220000-5,0.021279,45450000-6,0.020546,45230000-8,0.016233
8,1.0,UA-P-2020-04-27-004367-b,Нагляд за будівельними роботами ТН на капремо...,71247000-1 Нагляд за будівельними роботами,71247000-1,0.390795,45453000-7,0.174483,45000000-7,0.152054,71520000-9,0.077016,45220000-5,0.039711,99999999-9,0.024870,45450000-6,0.022052,71000000-8,0.019270,45233000-9,0.014037,45230000-8,0.012545
9,1.0,UA-P-2020-04-27-004362-b,Нагляд за будівельними роботами ТН на капремо...,71247000-1 Нагляд за будівельними роботами,71247000-1,0.326271,45453000-7,0.187396,45000000-7,0.143045,71520000-9,0.130678,71248000-8,0.031905,71240000-2,0.024127,99999999-9,0.021558,45220000-5,0.021279,45450000-6,0.020546,45230000-8,0.016233


In [ ]:
df1[['acc1', 'acc2','acc3', 'acc4', 'acc5','acc6', 'acc7', 'acc8','acc9', 'acc10']].sum()

acc1     25.768921
acc2      6.826635
acc3      4.326125
acc4      2.695172
acc5      1.673948
acc6      1.178411
acc7      0.879821
acc8      0.723703
acc9      0.596669
acc10     0.492114
dtype: float64

Pandas: sum DataFrame rows for given columns

[link text](https://stackoverflow.com/questions/25748683/pandas-sum-dataframe-rows-for-given-columns)

df['e'] = df.sum(axis=1)

df

```
# This is formatted as code
```



In [ ]:
#df1['sum_acc'] = df1[['acc1', 'acc2','acc3', 'acc4', 'acc5','acc6', 'acc7', 'acc8','acc9', 'acc10']].sum(axis=1)

In [ ]:
df1.sort_values(by=['plans_with_code']) 

,plans_with_code,Ідентифікатор пункту плану,Пункт плану (розширений),CPV з плану,id1,acc1,id2,acc2,id3,acc3,id4,acc4,id5,acc5,id6,acc6,id7,acc7,id8,acc8,id9,acc9,id10,acc10
1,1.0,UA-P-2020-04-27-004544-b,Послуги: Поточний ремонт ливневої каналізаційн...,"45230000-8 Будівництво трубопроводів, ліній зв...",45230000-8,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,UA-P-2020-04-27-004503-b,ДК 021:2015 — 50112000-3 - Послуги з ремонту і...,50112000-3 Послуги з ремонту і технічного обсл...,50112000-3,0.989054,50110000-9,0.010863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,UA-P-2020-04-27-004445-b,ДК 021:2015: 33631600-8 — Антисептичні та дезі...,33631600-8 Антисептичні та дезінфекційні засоби,33631600-8,0.749058,33630000-5,0.178217,24450000-3,0.016794,33631000-2,0.016722,33600000-6,0.014605,24455000-8,0.013877,33741300-9,0.002731,33740000-9,0.002706,33000000-0,0.001123,24400000-8,0.000936
7,1.0,UA-P-2020-04-27-004370-b,ДК 021:2015-71247000-1-Нагляд за будівельними ...,71247000-1 Нагляд за будівельними роботами,71247000-1,0.524111,45000000-7,0.171435,71520000-9,0.106359,45453000-7,0.087047,71248000-8,0.023481,45450000-6,0.010049,71240000-2,0.009743,71000000-8,0.009357,45220000-5,0.006557,45230000-8,0.005476
8,1.0,UA-P-2020-04-27-004367-b,ДК 021:2015-71247000-1-Нагляд за будівельними ...,71247000-1 Нагляд за будівельними роботами,71247000-1,0.564693,45000000-7,0.180882,71520000-9,0.078238,45453000-7,0.073209,71000000-8,0.011767,44111000-1,0.010573,45220000-5,0.009062,45450000-6,0.008514,45262000-1,0.007294,45233000-9,0.006562
9,1.0,UA-P-2020-04-27-004362-b,ДК 021:2015-71247000-1-Нагляд за будівельними ...,71247000-1 Нагляд за будівельними роботами,71247000-1,0.524111,45000000-7,0.171435,71520000-9,0.106359,45453000-7,0.087047,71248000-8,0.023481,45450000-6,0.010049,71240000-2,0.009743,71000000-8,0.009357,45220000-5,0.006557,45230000-8,0.005476
10,1.0,UA-P-2020-04-27-004356-b,ДК 021:2015-71247000-1-Нагляд за будівельними ...,71247000-1 Нагляд за будівельними роботами,71247000-1,0.504874,71520000-9,0.145245,45453000-7,0.142320,45000000-7,0.087701,71248000-8,0.020501,71240000-2,0.012665,45262600-7,0.011161,45450000-6,0.009976,45220000-5,0.008857,71000000-8,0.008522
11,1.0,UA-P-2020-04-27-004228-b,Послуги: топографо-геодезичні та картографічні...,71355000-1 Геодезичні послуги,71354000-4,0.245492,71355200-3,0.220746,71350000-6,0.196833,71222200-2,0.136232,71351810-4,0.091788,71355000-1,0.048566,71250000-5,0.021309,71340000-3,0.007491,71410000-5,0.006957,71354100-5,0.004683
0,NaN,UA-P-2020-04-28-000015-b,інженерно-геологічних вишукувань по об’єкту: «...,71510000-6 Послуги у сфері інженерно-геологічн...,71510000-6,0.260197,71000000-8,0.103378,45000000-7,0.082351,71240000-2,0.058228,71350000-6,0.057840,45454000-4,0.053863,71250000-5,0.046817,71320000-7,0.042009,45200000-9,0.028162,71355000-1,0.024628
2,NaN,UA-P-2020-04-27-004528-b,"""Капітальний ремонт даху з водовідведенням у ...",71520000-9 Послуги з нагляду за виконанням буд...,45453000-7,0.412781,45450000-6,0.153498,45261000-4,0.152376,45260000-7,0.059486,45200000-9,0.047588,45400000-1,0.034377,45210000-2,0.025401,45230000-8,0.012260,45300000-0,0.012021,45000000-7,0.009533


In [ ]:
df1.shape

(15384, 24)

In [ ]:
len(df1['Ідентифікатор пункту плану'].unique())

15384

In [ ]:
from pandas import ExcelWriter

writer = ExcelWriter(GDRIVE_PATH+'ocds_analytics_lviv_data_raw.xlsx')
df1.to_excel(writer,'Sheet1', index=False)
writer.save()

NameError: ignored

In [ ]:
df1.to_excel?

In [ ]:
df1.to_pickle(GDRIVE_PATH+"out_data.pkl")

In [ ]:
#load Excel file 

df = pd.read_excel(io=GDRIVE_PATH+'ocds_analytics_lviv_data_raw.xlsx', sheet_name='Sheet1')
test_frame = df

In [ ]:
for column in ['id1', 'id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8', 'id9', 'id10']:
    test_frame[column] = test_frame[column].replace('\d{4}\D\d{1}$', '', regex=True)

Define a dictionary containing students data 
```
data = {'Name': ['Ankit', 'Amit', 'Aishwarya', 'Priyanka'], 
                'Age': [21, 19, 20, 18], 
                'Stream': ['Math', 'Commerce', 'Arts', 'Biology'], 
                'Percentage': [88, 92, 95, 70]}

# Convert the dictionary into DataFrame 
df = pd.DataFrame(data, columns = ['Name', 'Age', 'Stream', 'Percentage']) 

print("Given Dataframe :\n", df) 
  
print("\nIterating over rows using loc function :\n") 

# iterate through each row and select  
# 'Name' and 'Age' column respectively. 
for i in range(len(df)) : 
  print(df.loc[i, "Name"], df.loc[i, "Age"])
```

  

In [ ]:
def unique_ids(row):
    ids = []
    accs = []
    for i in range(1, 11):
        id = row['id'+str(i)]
        acc = row['acc'+str(i)]
        if id in ids:
            continue
        ids.append(id)
        accs.append(acc)
        if len(ids) == 3:
            break
    res = {'uid1': ids[0], 'acc1': accs[0]}
    if len(ids) > 1:
        res['uid2'] = ids[1]
        res['acc2'] = accs[1]
    if len(ids) > 2:
        res['uid3'] = ids[2]
        res['acc3'] = accs[2]
    return pd.Series(res)

In [ ]:
df2 = test_frame.apply(unique_ids, axis=1)

In [ ]:
df2[['plans_with_code', 'Ідентифікатор пункту плану',	'Пункт плану (розширений)', 'CPV з плану']] = \
    test_frame[['plans_with_code', 'Ідентифікатор пункту плану',	'Пункт плану (розширений)', 'CPV з плану']]

In [ ]:
df2.shape

(15384, 10)

In [ ]:
cols = ['plans_with_code', 'Ідентифікатор пункту плану', 'Пункт плану (розширений)', 'CPV з плану',]

In [ ]:
for i in range(1, 4):
    cols.extend(['uid'+str(i), 'acc'+str(i)]) #'desc'+str(i), 

print(cols)

['plans_with_code', 'Ідентифікатор пункту плану', 'Пункт плану (розширений)', 'CPV з плану', 'uid1', 'acc1', 'uid2', 'acc2', 'uid3', 'acc3']


In [ ]:
df2 = df2[cols]

In [ ]:
df2.head(5)

,plans_with_code,Ідентифікатор пункту плану,Пункт плану (розширений),CPV з плану,uid1,acc1,uid2,acc2,uid3,acc3
0,NaN,UA-P-2020-04-28-000015-b,інженерногеологічних вишукувань по об’єкту «Ре...,71510000-6 Послуги у сфері інженерно-геологічн...,7971,0.244532,7124,0.045981,4545,0.044919
1,1.0,UA-P-2020-04-27-004544-b,Послуги Поточний ремонт ливневої каналізаційно...,"45230000-8 Будівництво трубопроводів, ліній зв...",4523,0.999917,NaN,NaN,NaN,NaN
2,NaN,UA-P-2020-04-27-004528-b,"""Капітальний ремонт даху з водовідведенням у ...",71520000-9 Послуги з нагляду за виконанням буд...,4545,0.412781,4526,0.152376,4520,0.047588
3,1.0,UA-P-2020-04-27-004503-b,Послуги з ремонту і технічного обслуговува...,50112000-3 Послуги з ремонту і технічного обсл...,5011,0.517599,5053,0.018717,5010,0.004024
4,NaN,UA-P-2020-04-27-004470-b,"""Ремонтнореставраційні роботи будинку на вул. ...",45453000-7 Капітальний ремонт і реставрація,4526,0.197195,4523,0.151064,4531,0.091480


In [ ]:
03-44 -4 i не 336
45 - 2
others and 336 - 3

In [ ]:
def match_level(row):
    cpv = row["CPV з плану"]

    if cpv.startswith('45'):
        cpv = cpv[:2]
    elif not cpv.startswith('336') and (cpv > '03' and cpv < '45'):
        cpv = cpv[:4]
    else:
        cpv = cpv[:3]
    try:
        int(cpv)
    except ValueError:
        print('Підозрілий cpv ', cpv)

    if str(row['uid1']).startswith(cpv):
       return 1
    elif str(row['uid2']).startswith(cpv):
       return 2
    elif str(row['uid3']).startswith(cpv):
       return 3
    else:
       return 0

In [ ]:
'03' < '03a'

True

In [ ]:
#example
#pd.Series(['a1', 'b2', 'c3'], dtype="string").str.extract(r'[ab](\d)', expand=True)

In [ ]:
 level = df2.apply(match_level, axis=1)
 type(level)

pandas.core.series.Series

In [ ]:
df2['Рівень збігу'] = level
df2.head(10)

,plans_with_code,Ідентифікатор пункту плану,Пункт плану (розширений),CPV з плану,uid1,acc1,uid2,acc2,uid3,acc3,Рівень збігу
0,NaN,UA-P-2020-04-28-000015-b,інженерногеологічних вишукувань по об’єкту «Ре...,71510000-6 Послуги у сфері інженерно-геологічн...,7971,0.244532,7124,0.045981,4545,0.044919,0
1,1.0,UA-P-2020-04-27-004544-b,Послуги Поточний ремонт ливневої каналізаційно...,"45230000-8 Будівництво трубопроводів, ліній зв...",4523,0.999917,NaN,NaN,NaN,NaN,1
2,NaN,UA-P-2020-04-27-004528-b,"""Капітальний ремонт даху з водовідведенням у ...",71520000-9 Послуги з нагляду за виконанням буд...,4545,0.412781,4526,0.152376,4520,0.047588,0
3,1.0,UA-P-2020-04-27-004503-b,Послуги з ремонту і технічного обслуговува...,50112000-3 Послуги з ремонту і технічного обсл...,5011,0.517599,5053,0.018717,5010,0.004024,1
4,NaN,UA-P-2020-04-27-004470-b,"""Ремонтнореставраційні роботи будинку на вул. ...",45453000-7 Капітальний ремонт і реставрація,4526,0.197195,4523,0.151064,4531,0.091480,1
5,1.0,UA-P-2020-04-27-004445-b,Антисептичні та дезінфекційні засоби,33631600-8 Антисептичні та дезінфекційні засоби,3363,0.420096,3360,0.109936,2445,0.107754,1
6,NaN,UA-P-2020-04-27-004398-b,Металопластикові вироби,"44221000-5 Вікна, двері та супутні вироби",4422,0.380415,4421,0.088449,1952,0.063580,1
7,1.0,UA-P-2020-04-27-004370-b,Нагляд за будівельними роботами ТН на капремо...,71247000-1 Нагляд за будівельними роботами,7124,0.326271,4545,0.187396,4500,0.143045,1
8,1.0,UA-P-2020-04-27-004367-b,Нагляд за будівельними роботами ТН на капремо...,71247000-1 Нагляд за будівельними роботами,7124,0.390795,4545,0.174483,4500,0.152054,1
9,1.0,UA-P-2020-04-27-004362-b,Нагляд за будівельними роботами ТН на капремо...,71247000-1 Нагляд за будівельними роботами,7124,0.326271,4545,0.187396,4500,0.143045,1


In [ ]:
level.value_counts(normalize=True)

1    0.694423
0    0.168812
2    0.093409
3    0.043357
dtype: float64

In [ ]:
from pandas import ExcelWriter

writer = ExcelWriter(GDRIVE_PATH+'ocds_analytics_lviv_data_updated.xlsx')
df2.to_excel(writer,'Sheet1', index=False)
writer.save()

## OCDS Analytics GraphQL API

In [ ]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
from json import dumps
base_url = 'https://ocdsanalytics.com/ua/prozorro/graphql'

with open('./graphql_token') as f:
    auth_token = f.read().strip()

transport_ocds = RequestsHTTPTransport(url = base_url, 
                                       headers = {'Authorization':'Bearer '+auth_token}, 
                                       use_json=True)

client = Client(transport=transport_ocds)

FileNotFoundError: ignored

In [ ]:
params = {'limit':1} 
query = gql('''
    query someVariables($limit:Int!) {
      Predictions {
        Unit(
          page: {limit: $limit},
          filters:[
            {eq: {field: "tender.title", value: "Системи та пристрої нагляду та охорони (Двоканальний автомобільний відеореєстратор)"}},
            {eq: {field: "tender.description", value: "Системи та пристрої нагляду та охорони (Двоканальний автомобільний відеореєстратор)"}},
            {eq: {field: "item.description", value: "Двоканальний автомобільний відеореєстратор"}},
            {eq: {field: "item.classification.id", value: "35120000-1"}},
            {gte: {field: "probability", value: "0.02"}}
          ]){
          values{
            entity{
              id
              name
              symbol
            }
            probability
          }
        }
      }
    }

''')
print(dumps(client.execute(query, variable_values=params), indent= 4))


{
    "Predictions": {
        "Unit": {
            "values": [
                {
                    "entity": {
                        "id": "H87",
                        "name": "\u0448\u0442\u0443\u043a\u0438",
                        "symbol": "\u0448\u0442."
                    },
                    "probability": 0.57209986
                }
            ]
        }
    }
}


In [ ]:
import gql.dsl as gqld

In [ ]:
q = gqld.DSLField?

In [ ]:
q = gqld.DSLField

In [ ]:
q.variable_definitions

[VariableDefinition(variable=Variable(name=Name(value='limit')), type=NonNullType(type=NamedType(name=Name(value='Int'))), default_value=None)]

In [ ]:
query = gql('''
    {
      Predictions {
        Classification(
          page: {limit: 3},
          filters:[
            {eq: {field: "tender.title", value: "Плитка керамічна"}},
            {eq: {field: "tender.description", value: "плитка керамічна"}},
            {eq: {field: "item.description", value: "Плитка керамічна Gress Атем Veneto, для підлоги, розмір – 400*400, колір – бежева, поверхня глазурована, матова"}},
            {eq: {field: "item.unit.id", value: "MTK"}},
            {gte: {field: "probability", value: "0.0002"}}
          ]){
          values{
            entity{
              id
              description
              scheme
            }
            probability
          }
        }
      }
    }
''')
print(dumps(client.execute(query, variable_values=params), indent= 4))

{
    "Predictions": {
        "Classification": {
            "values": [
                {
                    "entity": {
                        "id": "44111700-8",
                        "description": "\u041a\u0430\u0445\u0435\u043b\u044c",
                        "scheme": "CPV"
                    },
                    "probability": 0.6110372
                },
                {
                    "entity": {
                        "id": "44111300-4",
                        "description": "\u041a\u0435\u0440\u0430\u043c\u0456\u0447\u043d\u0456 \u0432\u0438\u0440\u043e\u0431\u0438",
                        "scheme": "CPV"
                    },
                    "probability": 0.16699347
                },
                {
                    "entity": {
                        "id": "44110000-4",
                        "description": "\u041a\u043e\u043d\u0441\u0442\u0440\u0443\u043a\u0446\u0456\u0439\u043d\u0456 \u043c\u0430\u0442\u0435\u0440\u0456\u0430\u043b\u0438",